In [6]:
import os
import pickle
import numpy as np
import pandas as pd
from collections import Counter

# 데이터셋 경로 설정
keypoints_dir = r"C:\Users\SongYoengEun\Desktop\cap\dataset\aist_plusplus_final\keypoints3d"  # 입력 폴더
save_path = r"C:\Users\SongYoengEun\Desktop\cap\Motion_Metadata_with_Emotions.csv"            # 결과 저장 파일

# 분석에 사용할 관절 인덱스 정의
left_arm = [5, 7, 9]
right_arm = [6, 8, 10]
pelvis_indices = [11, 12]

results = []  # 결과 저장 리스트

# 3D 포즈에서 특징 추출 함수
def extract_features(pose3d):
    # 팔 높이 변화량 계산 (Y축 평균값의 시간 변화)
    arm_y = pose3d[:, left_arm + right_arm, 1].mean(axis=1)
    if len(arm_y) > 30:
        arm_delta = arm_y[30:] - arm_y[:-30]
        avg_arm_delta = arm_delta.mean()
    else:
        avg_arm_delta = 0

    # 포즈 속도: 프레임 간 관절 이동량의 평균
    speed = np.linalg.norm(np.diff(pose3d, axis=0), axis=2).mean()

    # pelvis 이동량: 중심 좌표 이동량의 평균
    pelvis_center = pose3d[:, pelvis_indices, :].mean(axis=1)
    pelvis_shift = np.linalg.norm(np.diff(pelvis_center, axis=0), axis=1).mean()

    # 신체 퍼짐도: 각 프레임에서 관절 간 거리의 표준편차 평균
    spread = []
    for frame in pose3d:
        dists = [np.linalg.norm(frame[i] - frame[j]) for i in range(17) for j in range(i+1, 17)]
        spread.append(np.std(dists))
    avg_spread = np.mean(spread)

    return avg_arm_delta, speed, pelvis_shift, avg_spread

# 감정 예측 함수 (규칙 기반)
def predict_emotions(avg_arm_delta, speed, pelvis_shift, avg_spread):
    emotions = []

    # 팔 높이 변화 → 자신감 or 슬픔
    if avg_arm_delta > 15:
        emotions += ["자신감", "행복"]
    elif avg_arm_delta < -15:
        emotions += ["슬픔"]

    # 속도 → 열정, 희망, 차분함
    if speed > 20:
        emotions += ["열정"]
    elif speed > 12:
        emotions += ["희망"]
    elif speed < 8:
        emotions += ["차분함"]

    # 중심 이동량 → 도전 or 차분함/슬픔
    if pelvis_shift > 10:
        emotions += ["도전"]
    elif pelvis_shift < 5:
        emotions += ["슬픔", "차분함"]

    # 퍼짐도 → 매혹 or 공포
    if avg_spread > 140:
        emotions += ["매혹"]
    elif avg_spread < 90:
        emotions += ["공포"]

    # 등장 빈도 기준 상위 2개 감정 선택
    emotion_counter = Counter(emotions)
    top_emotions = [e for e, _ in emotion_counter.most_common(2)]
    return top_emotions

# 폴더 내 모든 pkl 파일에 대해 반복 수행
for filename in os.listdir(keypoints_dir):
    if not filename.endswith(".pkl"):
        continue

    motion_id = filename.replace(".pkl", "")

    # pkl 파일 로드
    with open(os.path.join(keypoints_dir, filename), "rb") as f:
        data = pickle.load(f)
        if "keypoints3d_optim" not in data:
            continue  # 최적화된 포즈 정보 없으면 건너뜀
        pose3d = data["keypoints3d_optim"]

    # 특징 추출 및 감정 예측
    avg_arm_delta, speed, pelvis_shift, avg_spread = extract_features(pose3d)
    emotions = predict_emotions(avg_arm_delta, speed, pelvis_shift, avg_spread)

    # 결과 저장
    results.append({
        "MOTION_ID": motion_id,
        "EMOTION_1": emotions[0] if len(emotions) > 0 else None,
        "EMOTION_2": emotions[1] if len(emotions) > 1 else None,
        "avg_arm_delta": avg_arm_delta,
        "speed": speed,
        "pelvis_shift": pelvis_shift,
        "avg_spread": avg_spread
    })

# CSV 파일로 저장
if results:
    df = pd.DataFrame(results)
    df.to_csv(save_path, index=False, encoding="utf-8-sig")
    print("감정 예측 결과 저장 완료!")
    print(df.head())
else:
    print("분석 결과 없음!")


✅ 감정 예측 결과 저장 완료!
                    MOTION_ID EMOTION_1 EMOTION_2  avg_arm_delta     speed  \
0  gBR_sBM_cAll_d04_mBR0_ch01       차분함        슬픔       0.442163  1.190426   
1  gBR_sBM_cAll_d04_mBR0_ch02       차분함        슬픔       0.060897  1.730326   
2  gBR_sBM_cAll_d04_mBR0_ch03       차분함        슬픔      -0.184639  1.417298   
3  gBR_sBM_cAll_d04_mBR0_ch04       차분함        슬픔       0.050326  1.656661   
4  gBR_sBM_cAll_d04_mBR0_ch05       차분함        슬픔      -0.938572  1.717238   

   pelvis_shift  avg_spread  
0      0.778513   36.385095  
1      1.017101   34.046623  
2      1.030659   35.658800  
3      0.842656   33.999885  
4      0.981873   31.838545  


In [17]:
import os
import pickle
import numpy as np
import pandas as pd
from collections import defaultdict

# --- 설정 ---
keypoints_dir = r"C:\Users\SongYoengEun\Desktop\cap\dataset\aist_plusplus_final\keypoints3d"  # 3D 포즈 pkl 파일이 저장된 디렉토리
sample_size_per_genre = 140  # 각 장르당 분석할 최대 파일 수

# --- 파일 이름의 그룹 코드(gXX) → 장르 이름 매핑 ---
genre_map = {
    "gBR": "Breakdance",
    "gPO": "Pop",
    "gLO": "Lock",
    "gWA": "Waack",
    "gHO": "House",
    "gKR": "Krump",
    "gJS": "Jazz",
    "gLH": "LA_Hiphop",
    "gMH": "Middle_Hiphop",
    "gJB": "Ballet_Jazz"
}

# --- 관절 인덱스 정의 ---
left_arm = [5, 7, 9]       # 왼팔: shoulder, elbow, wrist
right_arm = [6, 8, 10]     # 오른팔
pelvis_indices = [11, 12]  # pelvis 기준 관절 2개

# --- 3D 포즈에서 특징 추출 함수 ---
def extract_features(pose3d):
    # 팔 높이 변화량 (시간 흐름에 따른 Y 좌표 평균값 차이)
    arm_y = pose3d[:, left_arm + right_arm, 1].mean(axis=1)
    if len(arm_y) > 30:
        arm_delta = arm_y[30:] - arm_y[:-30]  # 30프레임 간 비교
        avg_arm_delta = arm_delta.mean()
    else:
        avg_arm_delta = 0  # 프레임이 짧은 경우 0으로 처리

    # 포즈 속도: 프레임 간 관절 위치 변화량의 평균
    speed = np.linalg.norm(np.diff(pose3d, axis=0), axis=2).mean()

    # pelvis 이동량: 중심이 얼마나 움직였는지
    pelvis_center = pose3d[:, pelvis_indices, :].mean(axis=1)
    pelvis_shift = np.linalg.norm(np.diff(pelvis_center, axis=0), axis=1).mean()

    # 신체 퍼짐도: 관절 간 거리의 표준편차
    spread = []
    for frame in pose3d[::5]:  # 5프레임 간격으로 계산 (속도 개선)
        dists = [np.linalg.norm(frame[i] - frame[j]) for i in range(17) for j in range(i+1, 17)]
        spread.append(np.std(dists))
    avg_spread = np.mean(spread)

    return avg_arm_delta, speed, pelvis_shift, avg_spread

# --- 장르별 특징 저장을 위한 딕셔너리 초기화 ---
features_by_genre = defaultdict(list)
genre_file_count = defaultdict(int)

# --- 폴더 내 모든 pkl 파일 반복 처리 ---
for fname in os.listdir(keypoints_dir):
    if not fname.endswith(".pkl"):
        continue  # pkl 파일만 처리

    try:
        group_code = fname.split("_")[0]  # 파일명에서 gBR 등 그룹코드 추출
        genre = genre_map.get(group_code, "Unknown")
        if genre == "Unknown":
            continue  # 매핑되지 않는 장르는 제외

        if genre_file_count[genre] >= sample_size_per_genre:
            continue  # 장르별 최대 샘플 수 초과 시 무시

        fpath = os.path.join(keypoints_dir, fname)
        with open(fpath, "rb") as f:
            data = pickle.load(f)
            if "keypoints3d_optim" not in data:
                continue  # 최적화된 keypoint가 없으면 제외
            pose3d = data["keypoints3d_optim"]

        features = extract_features(pose3d)
        features_by_genre[genre].append(features)
        genre_file_count[genre] += 1

    except Exception as e:
        print(f"{fname} 처리 중 오류 발생: {e}")

# --- 장르별 평균값 계산 및 데이터프레임 정리 ---
summary = []
for genre, feats in features_by_genre.items():
    arr = np.array(feats)
    summary.append({
        "genre": genre,
        "avg_arm_delta": arr[:, 0].mean(),
        "avg_speed": arr[:, 1].mean(),
        "avg_pelvis_shift": arr[:, 2].mean(),
        "avg_spread": arr[:, 3].mean()
    })

df = pd.DataFrame(summary)
df = df.sort_values("genre").reset_index(drop=True)

# --- 최종 결과 출력 ---
df


,genre,avg_arm_delta,avg_speed,avg_pelvis_shift,avg_spread
0,Ballet_Jazz,-0.362089,2.406885,1.414895,37.221151
1,Breakdance,-0.614702,2.035039,1.290362,31.250078
2,House,0.144919,1.526811,1.042896,31.142968
3,Jazz,-0.153497,0.746935,0.578851,36.312215
4,Krump,-0.139854,1.403069,0.821360,35.008792
5,LA_Hiphop,0.091431,1.264540,0.826907,33.163316
6,Lock,-0.133697,1.019211,0.593587,32.681688
7,Middle_Hiphop,0.074800,1.863205,1.230241,34.928749
8,Pop,-0.165768,0.628398,0.413003,33.880570
9,Waack,-0.097068,1.128556,0.587278,34.720947


In [19]:
import os
import pickle
import numpy as np
import pandas as pd
from collections import Counter

# [1] 장르별 평균값 정의 (이 값들은 기존 데이터셋 분석 결과 기반)
genre_avg_df_updated = pd.DataFrame([
    {"genre": "Ballet_Jazz", "avg_arm_delta": -0.362089, "avg_speed": 2.406885, "avg_pelvis_shift": 1.414895, "avg_spread": 37.221151},
    {"genre": "Breakdance", "avg_arm_delta": -0.614702, "avg_speed": 2.035039, "avg_pelvis_shift": 1.290362, "avg_spread": 31.250078},
    {"genre": "House", "avg_arm_delta": 0.144919, "avg_speed": 1.526811, "avg_pelvis_shift": 1.042896, "avg_spread": 31.142968},
    {"genre": "Jazz", "avg_arm_delta": -0.153497, "avg_speed": 0.746935, "avg_pelvis_shift": 0.578851, "avg_spread": 36.312215},
    {"genre": "Krump", "avg_arm_delta": -0.139854, "avg_speed": 1.403069, "avg_pelvis_shift": 0.821360, "avg_spread": 35.008792},
    {"genre": "LA_Hiphop", "avg_arm_delta": 0.091431, "avg_speed": 1.264540, "avg_pelvis_shift": 0.826907, "avg_spread": 33.163316},
    {"genre": "Lock", "avg_arm_delta": -0.133697, "avg_speed": 1.019211, "avg_pelvis_shift": 0.593587, "avg_spread": 32.681688},
    {"genre": "Middle_Hiphop", "avg_arm_delta": 0.074800, "avg_speed": 1.863205, "avg_pelvis_shift": 1.230241, "avg_spread": 34.928749},
    {"genre": "Pop", "avg_arm_delta": -0.165768, "avg_speed": 0.628398, "avg_pelvis_shift": 0.413003, "avg_spread": 33.880570},
    {"genre": "Waack", "avg_arm_delta": -0.097068, "avg_speed": 1.128556, "avg_pelvis_shift": 0.587278, "avg_spread": 34.720947},
])

# [1-1] 그룹코드 → 장르명 매핑 (파일명 첫 부분 기준)
group_to_genre = {
    "gBR": "Breakdance", "gPO": "Pop", "gLO": "Lock", "gWA": "Waack",
    "gHO": "House", "gKR": "Krump", "gJS": "Jazz", "gLH": "LA_Hiphop",
    "gMH": "Middle_Hiphop", "gJB": "Ballet_Jazz"
}

# [2] 포즈에서 평균 특징 추출 (4가지)
def extract_features(pose3d):
    left_arm = [5, 7, 9]
    right_arm = [6, 8, 10]
    pelvis_indices = [11, 12]

    # 팔 높이 변화량 (Y좌표 기준)
    arm_y = pose3d[:, left_arm + right_arm, 1].mean(axis=1)
    if len(arm_y) > 30:
        arm_delta = arm_y[30:] - arm_y[:-30]
        avg_arm_delta = arm_delta.mean()
    else:
        avg_arm_delta = 0

    # 평균 속도
    speed = np.linalg.norm(np.diff(pose3d, axis=0), axis=2).mean()

    # pelvis 중심의 이동량
    pelvis_center = pose3d[:, pelvis_indices, :].mean(axis=1)
    pelvis_shift = np.linalg.norm(np.diff(pelvis_center, axis=0), axis=1).mean()

    # 신체 퍼짐도 (관절 간 거리의 표준편차 평균)
    spread = []
    for frame in pose3d[::5]:  # 성능 개선을 위해 5프레임 간격 샘플링
        dists = [np.linalg.norm(frame[i] - frame[j]) for i in range(17) for j in range(i + 1, 17)]
        spread.append(np.std(dists))
    avg_spread = np.mean(spread)

    return avg_arm_delta, speed, pelvis_shift, avg_spread

# [3] 감정 예측 함수 (장르 평균값 대비 비율로 판단)
def predict_emotions_from_pose(features, genre, genre_avg_df):
    emotions = []
    arm_delta, speed, pelvis_shift, spread = features
    base = genre_avg_df[genre_avg_df['genre'] == genre]
    if base.empty:
        return ["중립"]  # 장르 정보 없을 경우 중립 처리
    base = base.iloc[0]

    # 각 특징값이 평균 대비 얼마나 크거나 작은지에 따라 감정 부여
    if arm_delta > base["avg_arm_delta"] * 1.3:
        emotions += ["자신감", "열정"]
    elif arm_delta < base["avg_arm_delta"] * 0.7:
        emotions += ["슬픔", "분노"]

    if speed > base["avg_speed"] * 1.3:
        emotions += ["열정", "희망"]
    elif speed < base["avg_speed"] * 0.7:
        emotions += ["차분함", "슬픔"]

    if pelvis_shift > base["avg_pelvis_shift"] * 1.4:
        emotions += ["도전", "열정"]
    elif pelvis_shift < base["avg_pelvis_shift"] * 0.6:
        emotions += ["공포", "슬픔"]

    if spread > base["avg_spread"] * 1.3:
        emotions += ["매혹", "사랑"]
    elif spread < base["avg_spread"] * 0.7:
        emotions += ["공포", "혐오"]

    # 가장 많이 등장한 감정 2개 선택
    top_emotions = [e for e, _ in Counter(emotions).most_common(2)]
    return top_emotions or ["중립"]

# [4] pkl 파일 하나를 처리하는 함수
def infer_emotion_from_pkl(pkl_path, genre_avg_df):
    filename = os.path.basename(pkl_path)
    group_code = filename.split("_")[0]
    genre = group_to_genre.get(group_code, "Unknown")

    if genre == "Unknown":
        return ["Unknown genre"]

    with open(pkl_path, "rb") as f:
        data = pickle.load(f)
        if "keypoints3d_optim" not in data:
            return ["No pose data"]
        pose3d = data["keypoints3d_optim"]

    features = extract_features(pose3d)
    return predict_emotions_from_pose(features, genre, genre_avg_df)

# [5] 폴더 전체를 감정 분석하고 CSV로 저장
def batch_infer_emotions_to_csv(pkl_folder_path, genre_avg_df, output_csv_path):
    results = []

    for fname in os.listdir(pkl_folder_path):
        if not fname.endswith(".pkl"):
            continue

        fpath = os.path.join(pkl_folder_path, fname)
        try:
            emotions = infer_emotion_from_pkl(fpath, genre_avg_df)
            results.append({
                "filename": fname,
                "emotion_1": emotions[0] if len(emotions) > 0 else None,
                "emotion_2": emotions[1] if len(emotions) > 1 else None
            })
        except Exception as e:
            results.append({
                "filename": fname,
                "emotion_1": "Error",
                "emotion_2": str(e)
            })

    result_df = pd.DataFrame(results)
    result_df.to_csv(output_csv_path, index=False, encoding="utf-8-sig")
    print(f"감정 예측 결과 저장 완료! → {output_csv_path}")
    return result_df

# [6] 실행: 실제 디렉토리와 파일명에 맞게 설정
pkl_folder = r"C:\Users\SongYoengEun\Desktop\cap\dataset\aist_plusplus_final\keypoints3d"
output_csv = r"C:\Users\SongYoengEun\Desktop\cap\emotion_results.csv"
batch_infer_emotions_to_csv(pkl_folder, genre_avg_df_updated, output_csv)


✅ 감정 예측 결과 저장 완료! → C:\Users\SongYoengEun\Desktop\cap\emotion_results.csv


,filename,emotion_1,emotion_2
0,gBR_sBM_cAll_d04_mBR0_ch01.pkl,자신감,열정
1,gBR_sBM_cAll_d04_mBR0_ch02.pkl,자신감,열정
2,gBR_sBM_cAll_d04_mBR0_ch03.pkl,자신감,열정
3,gBR_sBM_cAll_d04_mBR0_ch04.pkl,자신감,열정
4,gBR_sBM_cAll_d04_mBR0_ch05.pkl,슬픔,분노
...,...,...,...
1403,gWA_sFM_cAll_d27_mWA2_ch17.pkl,자신감,열정
1404,gWA_sFM_cAll_d27_mWA2_ch21.pkl,자신감,열정
1405,gWA_sFM_cAll_d27_mWA3_ch18.pkl,자신감,열정
1406,gWA_sFM_cAll_d27_mWA4_ch19.pkl,자신감,열정


In [21]:
import os
import pickle
import numpy as np
import pandas as pd
from collections import Counter

# [1] 장르별 평균 + 표준편차 정의(Z_score 계산용 기)
genre_full_stats = pd.DataFrame([
    {"genre": "Ballet_Jazz", "avg_arm_delta": -0.362089, "avg_speed": 2.406885, "avg_pelvis_shift": 1.414895, "avg_spread": 37.221151, "std_arm_delta": 0.25, "std_speed": 0.6, "std_pelvis_shift": 0.4, "std_spread": 5.2},
    {"genre": "Breakdance", "avg_arm_delta": -0.614702, "avg_speed": 2.035039, "avg_pelvis_shift": 1.290362, "avg_spread": 31.250078, "std_arm_delta": 0.28, "std_speed": 0.7, "std_pelvis_shift": 0.45, "std_spread": 4.9},
    {"genre": "House", "avg_arm_delta": 0.144919, "avg_speed": 1.526811, "avg_pelvis_shift": 1.042896, "avg_spread": 31.142968, "std_arm_delta": 0.23, "std_speed": 0.5, "std_pelvis_shift": 0.38, "std_spread": 4.1},
    {"genre": "Jazz", "avg_arm_delta": -0.153497, "avg_speed": 0.746935, "avg_pelvis_shift": 0.578851, "avg_spread": 36.312215, "std_arm_delta": 0.21, "std_speed": 0.3, "std_pelvis_shift": 0.25, "std_spread": 5.0},
    {"genre": "Krump", "avg_arm_delta": -0.139854, "avg_speed": 1.403069, "avg_pelvis_shift": 0.821360, "avg_spread": 35.008792, "std_arm_delta": 0.26, "std_speed": 0.6, "std_pelvis_shift": 0.35, "std_spread": 4.5},
    {"genre": "LA_Hiphop", "avg_arm_delta": 0.091431, "avg_speed": 1.264540, "avg_pelvis_shift": 0.826907, "avg_spread": 33.163316, "std_arm_delta": 0.22, "std_speed": 0.4, "std_pelvis_shift": 0.3, "std_spread": 3.9},
    {"genre": "Lock", "avg_arm_delta": -0.133697, "avg_speed": 1.019211, "avg_pelvis_shift": 0.593587, "avg_spread": 32.681688, "std_arm_delta": 0.24, "std_speed": 0.45, "std_pelvis_shift": 0.3, "std_spread": 4.2},
    {"genre": "Middle_Hiphop", "avg_arm_delta": 0.074800, "avg_speed": 1.863205, "avg_pelvis_shift": 1.230241, "avg_spread": 34.928749, "std_arm_delta": 0.27, "std_speed": 0.55, "std_pelvis_shift": 0.4, "std_spread": 4.8},
    {"genre": "Pop", "avg_arm_delta": -0.165768, "avg_speed": 0.628398, "avg_pelvis_shift": 0.413003, "avg_spread": 33.880570, "std_arm_delta": 0.25, "std_speed": 0.35, "std_pelvis_shift": 0.28, "std_spread": 4.0},
    {"genre": "Waack", "avg_arm_delta": -0.097068, "avg_speed": 1.128556, "avg_pelvis_shift": 0.587278, "avg_spread": 34.720947, "std_arm_delta": 0.22, "std_speed": 0.4, "std_pelvis_shift": 0.3, "std_spread": 4.6},
])

# 파일명에서 장르 추출용 코드 매핑
group_to_genre = {
    "gBR": "Breakdance", "gPO": "Pop", "gLO": "Lock", "gWA": "Waack",
    "gHO": "House", "gKR": "Krump", "gJS": "Jazz", "gLH": "LA_Hiphop",
    "gMH": "Middle_Hiphop", "gJB": "Ballet_Jazz"
}

# [2] 포즈에서 4가지 특징 추출
def extract_features(pose3d):
    left_arm = [5, 7, 9]
    right_arm = [6, 8, 10]
    pelvis_indices = [11, 12]

    arm_y = pose3d[:, left_arm + right_arm, 1].mean(axis=1)
    if len(arm_y) > 30:
        arm_delta = arm_y[30:] - arm_y[:-30]
        avg_arm_delta = arm_delta.mean()
    else:
        avg_arm_delta = 0

    speed = np.linalg.norm(np.diff(pose3d, axis=0), axis=2).mean()
    pelvis_center = pose3d[:, pelvis_indices, :].mean(axis=1)
    pelvis_shift = np.linalg.norm(np.diff(pelvis_center, axis=0), axis=1).mean()

    spread = []
    for frame in pose3d[::5]:
        dists = [np.linalg.norm(frame[i] - frame[j]) for i in range(17) for j in range(i + 1, 17)]
        spread.append(np.std(dists))
    avg_spread = np.mean(spread)

    return avg_arm_delta, speed, pelvis_shift, avg_spread

# [3] Z-score 기반 감정 예측 (13가지 감정 대응)
def predict_emotions_full(features, genre, genre_stats_df):
    emotions = []
    arm_delta, speed, pelvis_shift, spread = features

    row = genre_stats_df[genre_stats_df["genre"] == genre]
    if row.empty:
        return ["Unknown"]
    row = row.iloc[0]
    
    # Z-score 계산
    z_arm = (arm_delta - row["avg_arm_delta"]) / row["std_arm_delta"]
    z_speed = (speed - row["avg_speed"]) / row["std_speed"]
    z_pelvis = (pelvis_shift - row["avg_pelvis_shift"]) / row["std_pelvis_shift"]
    z_spread = (spread - row["avg_spread"]) / row["std_spread"]

    # 감정 조건별 분류 (z값 조합으로 예측)
    if z_arm > 1.2 and z_speed > 1.0:
        emotions.append("행복")
    if z_arm < -1.0:
        emotions.append("슬픔")
    if z_arm < -1.0 and z_speed > 1.0:
        emotions.append("분노")
    if z_pelvis < -1.0 and z_spread < -1.2:
        emotions.append("혐오")
    if z_pelvis < -1.0:
        emotions.append("공포")
    if z_spread > 1.0 and z_pelvis > 1.0:
        emotions.append("놀람")
    if z_spread > 1.2 and z_pelvis < 0.5:
        emotions.append("사랑")
    if z_speed > 0.8 and z_arm > 0.5:
        emotions.append("희망")
    if z_speed > 1.0:
        emotions.append("열정")
    if z_arm > 1.0:
        emotions.append("자신감")
    if z_spread > 1.2:
        emotions.append("매혹")
    if z_pelvis > 1.0:
        emotions.append("도전")
    if z_speed < -1.0:
        emotions.append("차분함")

    top_emotions = [e for e, _ in Counter(emotions).most_common(2)]
    return top_emotions or ["중립"]

# [4] 단일 파일 처리리
def infer_emotion_from_pkl_full(pkl_path, genre_stats_df):
    filename = os.path.basename(pkl_path)
    group_code = filename.split("_")[0]
    genre = group_to_genre.get(group_code, "Unknown")

    if genre == "Unknown":
        return ["Unknown genre"]

    with open(pkl_path, "rb") as f:
        data = pickle.load(f)
        if "keypoints3d_optim" not in data:
            return ["No pose data"]
        pose3d = data["keypoints3d_optim"]

    features = extract_features(pose3d)
    return predict_emotions_full(features, genre, genre_stats_df)

# [5] 전체 폴더 감정 예측 후 CSV 저장
def batch_infer_emotions_to_csv_full(pkl_folder_path, genre_stats_df, output_csv_path):
    results = []

    for fname in os.listdir(pkl_folder_path):
        if not fname.endswith(".pkl"):
            continue

        fpath = os.path.join(pkl_folder_path, fname)
        try:
            emotions = infer_emotion_from_pkl_full(fpath, genre_stats_df)
            results.append({
                "filename": fname,
                "emotion_1": emotions[0] if len(emotions) > 0 else None,
                "emotion_2": emotions[1] if len(emotions) > 1 else None
            })
        except Exception as e:
            results.append({
                "filename": fname,
                "emotion_1": "Error",
                "emotion_2": str(e)
            })

    result_df = pd.DataFrame(results)
    result_df.to_csv(output_csv_path, index=False, encoding="utf-8-sig")
    print(f"감정 예측 완료! → {output_csv_path}")
    return result_df

# [6] 실행 경로 설정
pkl_folder_path = r"C:\Users\SongYoengEun\Desktop\cap\dataset\aist_plusplus_final\keypoints3d"
output_csv_path = r"C:\Users\SongYoengEun\Desktop\emotion_results_full.csv"

batch_infer_emotions_to_csv_full(pkl_folder_path, genre_full_stats, output_csv_path)


✅ 감정 예측 완료! → C:\Users\SongYoengEun\Desktop\emotion_results_full.csv


,filename,emotion_1,emotion_2
0,gBR_sBM_cAll_d04_mBR0_ch01.pkl,공포,자신감
1,gBR_sBM_cAll_d04_mBR0_ch02.pkl,자신감,None
2,gBR_sBM_cAll_d04_mBR0_ch03.pkl,자신감,None
3,gBR_sBM_cAll_d04_mBR0_ch04.pkl,자신감,None
4,gBR_sBM_cAll_d04_mBR0_ch05.pkl,슬픔,None
...,...,...,...
1403,gWA_sFM_cAll_d27_mWA2_ch17.pkl,자신감,None
1404,gWA_sFM_cAll_d27_mWA2_ch21.pkl,중립,None
1405,gWA_sFM_cAll_d27_mWA3_ch18.pkl,중립,None
1406,gWA_sFM_cAll_d27_mWA4_ch19.pkl,중립,None
